In [1]:
import osmnx as ox
import mapbox
import pydeck as pdk
import plotly.express as px
mapbox_access_token = 'pk.eyJ1Ijoia2ltb3Nob25zIiwiYSI6ImNsYzkwcW9yazBpdjkzcHBjbWtlNW1iZHIifQ.Na_6glvmkjBpCFLJ7x1lYA'

# download the street network for a city
city_name = 'Nairobi, Kenya'
city_geocode = ox.geocode(city_name)
G = ox.graph_from_point(city_geocode,network_type='drive')
# plot the street network
#ox.plot_graph(G)
# convert graph to geojson file 
geojson = ox.graph_to_gdfs(G,

    nodes=False,edges=True)
geojson = geojson.to_json()



# create a Mapbox map using the GeoJSON object
# create a Mapbox map using the GeoJSON object
view_state = pdk.ViewState(latitude=city_geocode[1], longitude=city_geocode[0], zoom=10)
geo_data = pdk.Layer(type='GeoJsonLayer',geojson=geojson, mapbox_style='mapbox://styles/mapbox/streets-v9', opacity=0.8)
r = pdk.Deck(layers=[geo_data], initial_view_state=view_state,api_keys={'mapbox': mapbox_access_token})

# plot the map
# pdk.JupyterDeck(r)
r.show()


DeckGLWidget(carto_key=None, custom_libraries=[], google_maps_key=None, json_input='{\n  "initialViewState": {…

In [2]:
import ee
import geemap

In [19]:
# geemap.show_youtube('nAzZjKKd4w0')

In [9]:
js_snippet = """
// Load an image.
var image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318');

// Define the visualization parameters.
var vizParams = {
  bands: ['B5', 'B4', 'B3'],
  min: 0,
  max: 0.5,
  gamma: [0.95, 1.1, 1]
};

// Center the map and display the image.
Map.setCenter(-122.1899, 37.5010, 10); // San Francisco Bay
Map.addLayer(image, vizParams, 'false color composite');

"""

In [10]:
geemap.js_snippet_to_py(
    js_snippet, add_new_cell=True, import_ee=True, import_geemap=True, show_map=True
)

In [6]:
Map = geemap.Map()

# Load an image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

# Define the visualization parameters.
vizParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}


# (lon=36.931227857669064, lat=-0.1021642983885728)

# Center the map and display the image.
Map.setCenter(36.931227857669064, -0.1021642983885728, 7); # Nanyuki
Map.addLayer(image, vizParams, 'False color composite')

Map

Map(center=[-0.1021642983885728, 36.931227857669064], controls=(WidgetControl(options=['position', 'transparen…

In [12]:
map_js = """
function generate_collection(geometry) {
    var byMonth = ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY')
     .filter(ee.Filter.date(startDate, endDate))
     .select(['temperature_2m', 'total_precipitation'])
     .map(function (image) { 
       var reflBands = image.select('temperature_2m').subtract(273.15);
       image = image.addBands({
           srcImg: reflBands,
           overwrite: true
       });
       return image;
       
     })
     .map(function (image) {
       
       //https://confluence.ecmwf.int/pages/viewpage.action?pageId=197702790
       //calculation follows the link above: ERA5 monthly averaged reanalysis
       
       var month_start = ee.Date(image.get('system:time_start'));
       var month_end = month_start.advance(1,'months');
       var delta =  month_end.difference(month_start, 'days');
       
       var reflBands = image.select('total_precipitation').multiply(1000).multiply(delta);
       image = image.addBands({
           srcImg: reflBands,
           overwrite: true
       });
       return image;
     })
     .map(function (image) { return image.set('system:time_start', image.get('system:time_start')) });
 
   return byMonth;
   """

In [13]:
geemap.js_snippet_to_py(map_js)

In [7]:
Map = geemap.Map()

def generate_collection(geometry):
    byMonth = ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY') \
     .filter(ee.Filter.date('2020-01-01','2020-12-31')) \
     .select(['temperature_2m', 'total_precipitation'])

def func_wom (image):
       reflBands = image.select('temperature_2m').subtract(273.15)
       image = image.addBands({
           'srcImg': reflBands,
           'overwrite': True
       })
       return image \
     .map(func_wom)

def func_khn (image):

       #https:#confluence.ecmwf.int/pages/viewpage.action?pageId=197702790
       #calculation follows the link above: ERA5 monthly averaged reanalysis

       month_start = ee.Date(image.get('system:time_start'))
       month_end = month_start.advance(1,'months')
       delta =  month_end.difference(month_start, 'days')

       reflBands = image.select('total_precipitation').multiply(1000).multiply(delta)
       image = image.addBands({
           'srcImg': reflBands,
           'overwrite': True
       })
       return image \
     .map(func_khn)

# def func_zhk (image):
#   return image.set('system:time_start', image.get('system:time_start')) \
#      .map(function (image) ){ return image.set('system:time_start', image.get('system:time_start')) } \
#      .map(func_zhk)

#   #  return byMonth

ELEVATION

In [14]:
elevation_js = """
var elevation_dataset = ee.Image('NASA/NASADEM_HGT/001');
//var elevation_dataset = ee.Image('CGIAR/SRTM90_V4');

function get_elevation(geometry) {
  var elevation =  Math.round(elevation_dataset
    .reduceRegion(ee.Reducer.mean(), geometry, 100).get("elevation").getInfo());
  
  return elevation;
}
"""

In [15]:
geemap.js_snippet_to_py(elevation_js)

In [8]:
Map = geemap.Map()
import math

elevation_dataset = ee.Image('NASA/NASADEM_HGT/001')
#elevation_dataset = ee.Image('CGIAR/SRTM90_V4')

def get_elevation(geometry):
  elevation =  math.round(elevation_dataset \
    .reduceRegion(ee.Reducer.mean(), geometry, 100).get("elevation").getInfo())

  return elevation

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [9]:
control_js ="""
function control () {
  //define the left panel with some info and add it to the ui
  var panel = ui.Panel({
    style: { width: '400px' }
  })
  .add(ui.Label("Use drawing tool to define a region."))
  
  ui.root.add(panel);
  
  //define the reset button and add it to the map
  ...
  
  //define chart
  var temperature_chart;
  var elevation_string;

  function refresh(geometry) {
    Map.centerObject(geometry);
    panel.remove(temperature_chart);
    panel.remove(elevation_string);
    
    var byMonth = generate_collection(geometry);

    temperature_chart = generate_chart(byMonth, geometry);
    panel.add(temperature_chart);
    
    var elevation_value = get_elevation(geometry);
    var data = "Elevation: " + elevation_value + "m";
    elevation_string = ui.Label({value: data,  style: {textAlign: "center", width: '400px', fontSize: '30px', color: '484848'}} );
    
    panel.add(elevation_string);
  }
  
  //when the user redraw the region, refresh
  Map.drawingTools().onDraw(function (new_geometry) {
    geometry = new_geometry;
    refresh(geometry);
  })
  
  refresh(geometry);
}

control();
"""

In [18]:
geemap.js_snippet_to_py(control_js)

In [ ]:
Map = geemap.Map()

def control ():
  #define the left panel with some info and add it to the ui
  panel = ui.Panel({
    'style': '{ width': '400px' }
  }) \
  .add(ui.Label("Use drawing tool to define a region."))

  ui.root.add(panel) \
  ...

  #define chart
  # temperature_chart
  # elevation_string

  def refresh(geometry):
    Map.centerObject(geometry)
    panel.remove(temperature_chart)
    panel.remove(elevation_string)

    byMonth = generate_collection(geometry)

    temperature_chart = generate_chart(byMonth, geometry)
    panel.add(temperature_chart)

    elevation_value = get_elevation(geometry)
    data = "Elevation: " + elevation_value + "m"
    elevation_string = ui.Label({'value': data, 'style': '{textAlign': "center", 'width': '400px', 'fontSize': '30px', 'color': '484848'}} )

    panel.add(elevation_string)

  #when the user redraw the region, refresh
  Map.drawingTools().onDraw(function (new_geometry) {
    geometry =_geometry
    refresh(geometry)
  })

  refresh(geometry)

control()
Map